# Police and Incidents Clean-up and Connecting Police Incidents and School Locations

In [1]:
import warnings
import pandas as pd
from pathlib import Path
import geopandas

## Extract and Inspect Data

In [2]:
# File Path to Police Incidents in Philadelphia from January 2024- July 2024
incidents_data_to_load = Path("data\incidents_part1_part2.csv")

# Read Police Incidents into Pandas DataFrames
incidents_data = pd.read_csv(incidents_data_to_load)

# Check the DataFrame
incidents_data.head()

,the_geom,cartodb_id,the_geom_webmercator,objectid,dc_dist,psa,dispatch_date_time,dispatch_date,dispatch_time,hour,dc_key,location_block,ucr_general,text_general_code,point_x,point_y,lat,lng
0,0101000020E6100000439FB8B153CC52C0B6D18416A7F9...,3097150,0101000020110F00007AD170B738EE5FC136FD1FB6DC88...,13965196,18,3,2024-02-02 16:30:00+00,2024-02-02,11:30:00,11.0,2.024180e+11,300 BLOCK S 34TH ST,600,Thefts,-75.192608,39.950412,39.950412,-75.192608
1,NaN,3097416,NaN,17545922,17,1,2024-06-15 12:10:00+00,2024-06-15,08:10:00,8.0,2.024170e+11,2100 BLOCK SOUTH ST,600,Thefts,NaN,NaN,NaN,NaN
2,NaN,3097550,NaN,13065664,5,1,2024-01-06 20:35:00+00,2024-01-06,15:35:00,15.0,2.024050e+11,300 BLOCK ROCHELLE AV,600,Thefts,NaN,NaN,NaN,NaN
3,NaN,3098085,NaN,15737696,3,3,2024-04-26 21:13:00+00,2024-04-26,17:13:00,17.0,2.024030e+11,1100 BLOCK ELLSWORTH ST,1100,Fraud,NaN,NaN,NaN,NaN
4,0101000020E6100000B10E2EBA48C452C015DB8B059101...,3098523,0101000020110F0000117C9F5A8FE05FC117D42E37A291...,14865984,15,2,2024-03-15 11:17:00+00,2024-03-15,07:17:00,7.0,2.024150e+11,5500 BLOCK TULIP ST,600,Thefts,-75.066939,40.012238,40.012238,-75.066939


In [3]:
# Inspect Incident Data
incidents_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85529 entries, 0 to 85528
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   the_geom              79988 non-null  object 
 1   cartodb_id            85529 non-null  int64  
 2   the_geom_webmercator  79940 non-null  object 
 3   objectid              85529 non-null  int64  
 4   dc_dist               85529 non-null  int64  
 5   psa                   85495 non-null  object 
 6   dispatch_date_time    85529 non-null  object 
 7   dispatch_date         85529 non-null  object 
 8   dispatch_time         85529 non-null  object 
 9   hour                  85527 non-null  float64
 10  dc_key                85529 non-null  float64
 11  location_block        85513 non-null  object 
 12  ucr_general           85529 non-null  int64  
 13  text_general_code     85529 non-null  object 
 14  point_x               79940 non-null  float64
 15  point_y            

## Review of keys for data file by referencing the Open Philly Data website
#### 'cartodb_id' is the unique identifier for the crime incident record.
#### 'dc_dist' is the code for the police district responsible for the area where the incident took place.
#### 'text_general_code' provides a textual description of the incident type.
#### 'ucr_general' provides a standardized crime code.
#### 'location' specifies the geographic location of the incident.
#### 'psa' is Police Service Area within the Police District.  There are multiple service areas within the same district.

In [4]:
# Calculate the number of unique general codes and UCR codes for incident type
text_general_code_count = incidents_data['text_general_code'].nunique()
ucr_general_count = incidents_data['ucr_general'].nunique()

print("The number of unique general codes for incident type:", text_general_code_count)
print("The number of unique general UCR codes for incident type:", ucr_general_count)

# Group incidents by their UCR code and list them
ucr_incidents = incidents_data.groupby('ucr_general')['text_general_code'].unique()

# Convert the groupby object to a DataFrame for better readability
ucr_incidents_df = ucr_incidents.reset_index()
ucr_incidents_df.columns = ['ucr_general_code', 'incidents']

ucr_incidents_df

The number of unique general codes for incident type: 31
The number of unique general UCR codes for incident type: 26


,ucr_general_code,incidents
0,100,"[Homicide - Criminal, Homicide - Justifiable]"
1,200,[Rape]
2,300,"[Robbery Firearm, Robbery No Firearm]"
3,400,"[Aggravated Assault No Firearm, Aggravated Ass..."
4,500,"[Burglary Non-Residential, Burglary Residential]"
5,600,"[Thefts, Theft from Vehicle]"
6,700,[Motor Vehicle Theft]
7,800,[Other Assaults]
8,900,[Arson]
9,1000,[Forgery and Counterfeiting]


In [5]:
# Calculate the number of unique general codes and UCR codes for incident type
dc_dist_count = incidents_data['dc_dist'].nunique()
psa_count = incidents_data['psa'].nunique()
objectid_count = incidents_data['objectid'].nunique()
dc_key_count = incidents_data['dc_key'].nunique()

print("The number of police districts:", dc_dist_count)
print("The number of police service areas:", psa_count)
print("The number of object ids for incidents:", objectid_count)
print("The number dc_key_counts for incidents:", dc_key_count)

The number of police districts: 22
The number of police service areas: 6
The number of object ids for incidents: 85529
The number dc_key_counts for incidents: 85520


## Transforming Data

#### Duplicates: Although, there are duplicate 'dc_key's there is variation in the other data for the different unique 'object_id's. Because of this, the duplicate dc_keys for this dataset were not removed due to not knowing if they are actually duplicates.  Some of these same rows are missing coordinate data and will be removed due to their lack of coordinates needed for analysiss. There is a small number compared to total dataset, so this should not impact analysis.    


In [6]:
# Identify duplicates based on 'dc_key' 
duplicates = incidents_data[incidents_data.duplicated(subset=['dc_key'], keep=False)]

print("Duplicates based on 'dc_key':")
duplicates
# duplicates.count()

Duplicates based on 'dc_key':


,the_geom,cartodb_id,the_geom_webmercator,objectid,dc_dist,psa,dispatch_date_time,dispatch_date,dispatch_time,hour,dc_key,location_block,ucr_general,text_general_code,point_x,point_y,lat,lng
19063,0101000020E610000008F53E9F52C952C0B36AC658DC00...,3178665,0101000020110F00001B96BE5E1EE95FC1AC6968E0D990...,18326613,25,4,2024-07-14 03:12:00+00,2024-07-13,23:12:00,23.0,2.024250e+11,3500 BLOCK N WARNOCK ST,400,Aggravated Assault Firearm,-75.145668,40.006724,40.006724,-75.145668
20439,0101000020E6100000CBBAA03A2DC352C0B7A5C464E005...,3180345,0101000020110F00008CA0BECDADDE5FC1A866C1066A96...,15265578,15,3,2024-04-08 18:39:00+00,2024-04-08,14:39:00,14.0,2.024150e+11,7400 BLOCK REVERE ST,400,Aggravated Assault Firearm,-75.049636,40.045910,40.045910,-75.049636
29540,0101000020E6100000F7BD76FEE7C952C0605B3FFDE7FA...,3191206,0101000020110F00002770E7171CEA5FC18153AE41408A...,18383386,6,1,2024-04-07 04:00:00+00,2024-04-07,00:00:00,9.0,2.024060e+11,900 BLOCK HAMILTON ST,100,Homicide - Criminal,-75.154785,39.960205,39.960205,-75.154785
32818,NaN,3197371,NaN,18383324,2,2,2024-05-21 04:00:00+00,2024-05-21,00:00:00,14.0,2.024020e+11,6000 BLOCK BINGHAM ST,100,Homicide - Criminal,NaN,NaN,NaN,NaN
32819,NaN,3197372,NaN,18383325,2,2,2024-05-21 04:00:00+00,2024-05-21,00:00:00,14.0,2.024020e+11,6000 BLOCK BINGHAM ST,100,Homicide - Criminal,NaN,NaN,NaN,NaN
33071,NaN,3197847,NaN,18383336,16,3,2024-04-25 04:00:00+00,2024-04-25,00:00:00,17.0,2.024160e+11,2500 BLOCK A ST,100,Homicide - Criminal,NaN,NaN,NaN,NaN
33329,NaN,3198336,NaN,18383347,22,4,2024-02-29 05:00:00+00,2024-02-29,00:00:00,23.0,2.024220e+11,3800 BLOCK MT PLEASANT AVE,100,Homicide - Criminal,NaN,NaN,NaN,NaN
33333,NaN,3198341,NaN,18383350,15,3,2024-04-09 04:00:00+00,2024-04-09,00:00:00,15.0,2.024150e+11,7400 BLOCK REVERE ST,100,Homicide - Criminal,NaN,NaN,NaN,NaN
33349,NaN,3198358,NaN,18383346,22,4,2024-02-29 05:00:00+00,2024-02-29,00:00:00,23.0,2.024220e+11,3800 BLOCK MT PLEASANT DR,100,Homicide - Criminal,NaN,NaN,NaN,NaN
33356,NaN,3198365,NaN,18383363,25,4,2024-07-14 04:00:00+00,2024-07-14,00:00:00,23.0,2.024250e+11,1000 BLOCK W VENANGO ST,100,Homicide - Criminal,NaN,NaN,NaN,NaN


### Changing data to DateTime to be able to analyze Crime Incidents over time

In [7]:
# Convert 'dispatch_date_time' to datetime format
incidents_data['dispatch_date_time'] = pd.to_datetime(incidents_data['dispatch_date_time'], errors='coerce')

# Convert 'dispatch_date' to datetime format (date only)
incidents_data['dispatch_date'] = pd.to_datetime(incidents_data['dispatch_date'], format='%Y-%m-%d', errors='coerce')

# Convert 'dispatch_time' to datetime format (time only)
incidents_data['dispatch_time'] = pd.to_datetime(incidents_data['dispatch_time'], format='%H:%M:%S', errors='coerce').dt.time

# Check the DataFrame to ensure the conversion
print(incidents_data[['dispatch_date_time', 'dispatch_date', 'dispatch_time']].head())

incidents_data.info()

         dispatch_date_time dispatch_date dispatch_time
0 2024-02-02 16:30:00+00:00    2024-02-02      11:30:00
1 2024-06-15 12:10:00+00:00    2024-06-15      08:10:00
2 2024-01-06 20:35:00+00:00    2024-01-06      15:35:00
3 2024-04-26 21:13:00+00:00    2024-04-26      17:13:00
4 2024-03-15 11:17:00+00:00    2024-03-15      07:17:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85529 entries, 0 to 85528
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   the_geom              79988 non-null  object             
 1   cartodb_id            85529 non-null  int64              
 2   the_geom_webmercator  79940 non-null  object             
 3   objectid              85529 non-null  int64              
 4   dc_dist               85529 non-null  int64              
 5   psa                   85495 non-null  object             
 6   dispatch_date_time    85529 non-null  datetime

### Creating a Combined Key of the District Number and Police Service Area Number:  
#### This will allow us to locate the specific location of the Police Service Area and use the Key to connect location of other data. 

In [8]:
# Creating the combined key
incidents_data['district_psa'] = incidents_data['dc_dist'].astype(str).str.zfill(2) + incidents_data['psa'].astype(str)

incidents_data = incidents_data[incidents_data['district_psa']!='77A']

incidents_data = incidents_data.reset_index(drop=True)

# Display the result
incidents_data.head()

,the_geom,cartodb_id,the_geom_webmercator,objectid,dc_dist,psa,dispatch_date_time,dispatch_date,dispatch_time,hour,dc_key,location_block,ucr_general,text_general_code,point_x,point_y,lat,lng,district_psa
0,0101000020E6100000439FB8B153CC52C0B6D18416A7F9...,3097150,0101000020110F00007AD170B738EE5FC136FD1FB6DC88...,13965196,18,3,2024-02-02 16:30:00+00:00,2024-02-02,11:30:00,11.0,2.024180e+11,300 BLOCK S 34TH ST,600,Thefts,-75.192608,39.950412,39.950412,-75.192608,183
1,NaN,3097416,NaN,17545922,17,1,2024-06-15 12:10:00+00:00,2024-06-15,08:10:00,8.0,2.024170e+11,2100 BLOCK SOUTH ST,600,Thefts,NaN,NaN,NaN,NaN,171
2,NaN,3097550,NaN,13065664,5,1,2024-01-06 20:35:00+00:00,2024-01-06,15:35:00,15.0,2.024050e+11,300 BLOCK ROCHELLE AV,600,Thefts,NaN,NaN,NaN,NaN,051
3,NaN,3098085,NaN,15737696,3,3,2024-04-26 21:13:00+00:00,2024-04-26,17:13:00,17.0,2.024030e+11,1100 BLOCK ELLSWORTH ST,1100,Fraud,NaN,NaN,NaN,NaN,033
4,0101000020E6100000B10E2EBA48C452C015DB8B059101...,3098523,0101000020110F0000117C9F5A8FE05FC117D42E37A291...,14865984,15,2,2024-03-15 11:17:00+00:00,2024-03-15,07:17:00,7.0,2.024150e+11,5500 BLOCK TULIP ST,600,Thefts,-75.066939,40.012238,40.012238,-75.066939,152


#### Creating a New Data Frame Removing Columns that are not needed and Rows with missing data

In [9]:
#Make a new data frame for Incident Information and Removing Columns and Rows with NaN
incidents_data_df = pd.DataFrame({
    'incident_id': incidents_data['objectid'],
    'district': incidents_data['dc_dist'],
    'police_service_area': incidents_data['psa'], 
    'district_psa': incidents_data['district_psa'], 
    'dispatch_date_and_time': incidents_data['dispatch_date_time'],
    'dispatch_date': incidents_data['dispatch_date'],
    'dispatch_time': incidents_data['dispatch_time'],      
    'dispatch_hour': incidents_data['hour'],
    'location_block': incidents_data['location_block'],
    'ucr_code': incidents_data['ucr_general'],
    'general_crime_category': incidents_data['text_general_code']
})

incidents_data_df.head()

,incident_id,district,police_service_area,district_psa,dispatch_date_and_time,dispatch_date,dispatch_time,dispatch_hour,location_block,ucr_code,general_crime_category
0,13965196,18,3,183,2024-02-02 16:30:00+00:00,2024-02-02,11:30:00,11.0,300 BLOCK S 34TH ST,600,Thefts
1,17545922,17,1,171,2024-06-15 12:10:00+00:00,2024-06-15,08:10:00,8.0,2100 BLOCK SOUTH ST,600,Thefts
2,13065664,5,1,051,2024-01-06 20:35:00+00:00,2024-01-06,15:35:00,15.0,300 BLOCK ROCHELLE AV,600,Thefts
3,15737696,3,3,033,2024-04-26 21:13:00+00:00,2024-04-26,17:13:00,17.0,1100 BLOCK ELLSWORTH ST,1100,Fraud
4,14865984,15,2,152,2024-03-15 11:17:00+00:00,2024-03-15,07:17:00,7.0,5500 BLOCK TULIP ST,600,Thefts


In [10]:
# Drop rows with any NaN values
incidents_data_df = incidents_data_df.dropna(how='any')

# Drop rows where 'Not Available' is present in any column
incidents_data_df = incidents_data_df[~incidents_data_df.isin(['Not Available']).any(axis=1)]

columns_to_convert = [
    'incident_id',
    'district',
    'police_service_area',
    'district_psa',
    'dispatch_date_and_time',
    'dispatch_date',
    'dispatch_time',
    'dispatch_hour',
    'location_block',
    'ucr_code',
    'general_crime_category'
]

# Copy the original DataFrame
incidents_time_place_df = incidents_data_df.copy()  

incidents_time_place_df.head()

,incident_id,district,police_service_area,district_psa,dispatch_date_and_time,dispatch_date,dispatch_time,dispatch_hour,location_block,ucr_code,general_crime_category
0,13965196,18,3,183,2024-02-02 16:30:00+00:00,2024-02-02,11:30:00,11.0,300 BLOCK S 34TH ST,600,Thefts
1,17545922,17,1,171,2024-06-15 12:10:00+00:00,2024-06-15,08:10:00,8.0,2100 BLOCK SOUTH ST,600,Thefts
2,13065664,5,1,051,2024-01-06 20:35:00+00:00,2024-01-06,15:35:00,15.0,300 BLOCK ROCHELLE AV,600,Thefts
3,15737696,3,3,033,2024-04-26 21:13:00+00:00,2024-04-26,17:13:00,17.0,1100 BLOCK ELLSWORTH ST,1100,Fraud
4,14865984,15,2,152,2024-03-15 11:17:00+00:00,2024-03-15,07:17:00,7.0,5500 BLOCK TULIP ST,600,Thefts


#### Creating a DataFrame to just show the District and PSA for future use

In [11]:
district_psa_df=pd.DataFrame({
    'district_psa':incidents_time_place_df['district_psa'],
    'district':incidents_time_place_df['district'],
    'police_service_area':incidents_time_place_df['police_service_area']
})

# Sort by 'District' first and then by 'Police Service Area'
district_psa_df = district_psa_df.sort_values(by=['district', 'police_service_area'])

# Remove duplicates based on a specific column or combination of columns
district_psa_df = district_psa_df.drop_duplicates(subset=['district_psa'], keep='first')

# Reset the index if needed
district_psa_df.reset_index(drop=True, inplace=True)

# Save the cleaned DataFrame to a new CSV file
output_file_path = 'jupyter_outputs/District_PSA.csv'

# Save the DataFrame to a CSV file
district_psa_df.to_csv(output_file_path, index=False)

print(f"DataFrame saved to {output_file_path}")
district_psa_df.head()

DataFrame saved to jupyter_outputs/District_PSA.csv


,district_psa,district,police_service_area
0,011,1,1
1,012,1,2
2,021,2,1
3,022,2,2
4,023,2,3


### Creating Crime Categories for analysis with grouped data

In [12]:
# Create the 'incident_code_df' dataframe
incident_code_df=pd.DataFrame({
    'ucr_code':incidents_data['ucr_general'],
    'general_crime_category':incidents_data['text_general_code']
})

# Sort by 'UCR Code' first and then by 'General Crime Category'
incident_code_df = incident_code_df.sort_values(by=['ucr_code'])

# Reset index if you want a clean index
incident_code_df = incident_code_df.reset_index(drop=True)

incident_code_df.head()

,ucr_code,general_crime_category
0,100,Homicide - Criminal
1,100,Homicide - Criminal
2,100,Homicide - Criminal
3,100,Homicide - Criminal
4,100,Homicide - Criminal


In [13]:
# Define the crime groups
group_mapping = {
    'Violent_Crimes': [
        'Homicide - Criminal', 'Homicide - Justifiable', 'Rape', 'Robbery No Firearm', 
        'Robbery Firearm', 'Aggravated Assault Firearm', 'Aggravated Assault No Firearm', 
        'Other Assaults'
    ],
    'Property_Crimes': [
        'Burglary Residential', 'Burglary Non-Residential', 'Thefts', 
        'Theft from Vehicle', 'Motor Vehicle Theft', 'Arson', 
        'Forgery and Counterfeiting', 'Receiving Stolen Property', 
        'Vandalism/Criminal Mischief'
    ],
    'Public_Order_Crimes': [
        'Disorderly Conduct', 'Vagrancy/Loitering', 'Public Drunkenness', 
        'Liquor Law Violations', 'DRIVING UNDER THE INFLUENCE'
    ],
    'Drug_and_Vice_Crimes': [
        'Prostitution and Commercialized Vice', 'Narcotic / Drug Law Violations', 
        'Other Sex Offenses (Not Commercialized)', 'Gambling Violations'
    ],
    'Other_Crimes': [
        'Offenses Against Family and Children', 'Fraud', 'Embezzlement', 
        'Weapon Violations', 'All Other Offenses'
    ]
}

# Reverse the mapping to map each crime category to its group
crime_to_group = {crime: group for group, crimes in group_mapping.items() for crime in crimes}

# Create a new column for the crime group
incidents_time_place_df['crime_group'] = incidents_time_place_df['general_crime_category'].map(crime_to_group)

incidents_time_place_df.head()

,incident_id,district,police_service_area,district_psa,dispatch_date_and_time,dispatch_date,dispatch_time,dispatch_hour,location_block,ucr_code,general_crime_category,crime_group
0,13965196,18,3,183,2024-02-02 16:30:00+00:00,2024-02-02,11:30:00,11.0,300 BLOCK S 34TH ST,600,Thefts,Property_Crimes
1,17545922,17,1,171,2024-06-15 12:10:00+00:00,2024-06-15,08:10:00,8.0,2100 BLOCK SOUTH ST,600,Thefts,Property_Crimes
2,13065664,5,1,051,2024-01-06 20:35:00+00:00,2024-01-06,15:35:00,15.0,300 BLOCK ROCHELLE AV,600,Thefts,Property_Crimes
3,15737696,3,3,033,2024-04-26 21:13:00+00:00,2024-04-26,17:13:00,17.0,1100 BLOCK ELLSWORTH ST,1100,Fraud,Other_Crimes
4,14865984,15,2,152,2024-03-15 11:17:00+00:00,2024-03-15,07:17:00,7.0,5500 BLOCK TULIP ST,600,Thefts,Property_Crimes


#### IncidentsTimePlace file as output to be used in DataBase

In [14]:
# Make an output for the new dataset
output_file_path = 'jupyter_outputs/Incidents_Time_Place.csv'

# Save the DataFrame to a CSV file
incidents_time_place_df.to_csv(output_file_path, index=False)
print(f"DataFrame saved to {output_file_path}")
print("-" * 60)

incidents_time_place_df.info()

DataFrame saved to jupyter_outputs/Incidents_Time_Place.csv
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 84834 entries, 0 to 84885
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   incident_id             84834 non-null  int64              
 1   district                84834 non-null  int64              
 2   police_service_area     84834 non-null  object             
 3   district_psa            84834 non-null  object             
 4   dispatch_date_and_time  84834 non-null  datetime64[ns, UTC]
 5   dispatch_date           84834 non-null  datetime64[ns]     
 6   dispatch_time           84834 non-null  object             
 7   dispatch_hour           84834 non-null  float64            
 8   location_block          84834 non-null  object             
 9   ucr_code                84834 non-null  int64          

#### Incidents file as output to be used in DataBase

In [15]:
# Group by 'UCR Code' and aggregate unique values of 'General Crime Category' and 'Crime Group'
incidents_df = incidents_time_place_df.groupby('ucr_code').agg({'general_crime_category': lambda x: x.unique(), 'crime_group': lambda x: x.unique()})

# Convert the series to a DataFrame
incidents_df = incidents_df.reset_index()

# Make an output for the new dataset
output_file_path = 'jupyter_outputs/Incidents.csv'

# Save the DataFrame to a CSV file
incidents_df.to_csv(output_file_path, index=False)
print(f"DataFrame saved to {output_file_path}")
print("-" * 60)

incidents_df.info()

DataFrame saved to jupyter_outputs/Incidents.csv
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ucr_code                26 non-null     int64 
 1   general_crime_category  26 non-null     object
 2   crime_group             26 non-null     object
dtypes: int64(1), object(2)
memory usage: 752.0+ bytes


### Connecting the Police Service Area (PSA) coordinates which are polygons to the location of schools coordinate points
#### Two GeoJson Files Extracted: Philadelphia Schools Locations and Boundaries of the Police Services Areas in Philadelphia

In [16]:
import geopandas as gpd
from shapely.geometry import Point

# Load school locations
school_gdf = gpd.read_file('data\Schools.geojson')

# Load PSA boundaries
psa_gdf = gpd.read_file('data\Boundaries_PSA.geojson')

# Ensure both GeoDataFrames are in the same CRS
school_gdf = school_gdf.to_crs(epsg=4326)
psa_gdf = psa_gdf.to_crs(epsg=4326)

# Perform a spatial join to find which PSA polygon each school is in
school_with_psa_gdf = gpd.sjoin(school_gdf, psa_gdf, how='left', predicate='within')

# Extract relevant PSA and district information
school_with_psa_gdf['PSA_NUM'] = school_with_psa_gdf['PSA_NUM']
school_with_psa_gdf['DISTRICT'] = school_with_psa_gdf['DISTRICT__']

# Save the result to a new file or use it as needed
school_with_psa_gdf.to_file('jupyter_outputs/schools_with_psa_info.geojson', driver='GeoJSON')

school_with_psa_gdf.head()

,OBJECTID_left,AUN,SCHOOL_NUM,LOCATION_ID,SCHOOL_NAME,SCHOOL_NAME_LABEL,STREET_ADDRESS,ZIP_CODE,PHONE_NUMBER,ACTIVE,...,PSACOV_,PSACOV_ID,ID,DISTRICT__,PSA_NUM,OLD_SECTOR,DESCRIPT,Shape__Area,Shape__Length,DISTRICT
0,1,226519902.0,NaN,None,WEST CATHOLIC PREPARATORY HIGH SCHOOL,WEST PHILADELPHIA CATHOLIC HIGH SCHOOL,4501-17 CHESTNUT ST,19139,(215) 386-2244,O,...,None,None,40,18,183,"A,B,C,D,E,F,J",,8.544550e+06,13652.470162,18
1,2,226519382.0,NaN,None,ST THOMAS AQUINAS SCHOOL,ST THOMAS AQUINAS SCHOOL,1616 S 17TH ST,19145,(215) 334-0878,O,...,None,None,37,17,173,"A,B,D,E,I,O",,2.392552e+06,6409.564054,17
2,3,226519222.0,NaN,None,ST RAYMOND OF PENAFORT SCHOOL,ST RAYMOND SCHOOL,1330 VERNON RD,19150,(215) 548-1919,O,...,None,None,26,14,141,"A,C,F,K,N,Q,Y",,1.172253e+07,15338.858275,14
3,4,226519422.0,NaN,None,BLESSED TRINITY REGIONAL CATHOLIC SCHOOL,BLESSED TRINITY REGIONAL CATHOLIC SCHOOL,3033 LEVICK ST,19149,(215) 338-9797,O,...,None,None,31,15,152,"I,J,K,L,M",Unruh to Bridge - Roosevelt to Delaware,1.473769e+07,17143.738939,15
4,5,226519142.0,NaN,None,ST PETER THE APOSTLE SCHOOL,ST PETER THE APOSTLE SCHOOL,1009 N 5TH ST,19123,(215) 922-5958,O,...,None,None,49,26,262,"G,H,I,L,M,N",Norris St to Poplar St - Front St to 10th St,4.176330e+06,8352.638368,26


In [17]:
school_with_psa_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 491 entries, 0 to 490
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   OBJECTID_left      491 non-null    int64   
 1   AUN                451 non-null    float64 
 2   SCHOOL_NUM         286 non-null    float64 
 3   LOCATION_ID        328 non-null    object  
 4   SCHOOL_NAME        483 non-null    object  
 5   SCHOOL_NAME_LABEL  491 non-null    object  
 6   STREET_ADDRESS     491 non-null    object  
 7   ZIP_CODE           491 non-null    object  
 8   PHONE_NUMBER       490 non-null    object  
 9   ACTIVE             131 non-null    object  
 10  GRADE_LEVEL        490 non-null    object  
 11  GRADE_ORG          491 non-null    object  
 12  ENROLLMENT         329 non-null    float64 
 13  TYPE               491 non-null    int64   
 14  TYPE_SPECIFIC      491 non-null    object  
 15  geometry           491 non-null    geometry
 16  index

### Creating a School Location DataFrame showing its locaiton in a specific Police Service Area and Exporting the file for use in database 

In [18]:
# Rename the specified columns
school_with_psa_gdf = school_with_psa_gdf.rename(columns={
    'LOCATION_ID': 'school_id', 
    'SCHOOL_NAME': 'school_name', 
    'SCHOOL_NAME_LABEL': 'school_name_label',
    'STREET_ADDRESS': 'street_address',
    'ZIP_CODE': 'zip_code',
    'PHONE_NUMBER': 'phone_number',
    'GRADE_LEVEL': 'grade_level',
    'GRADE_ORG': 'grade_org',
    'TYPE': 'type',
    'TYPE_SPECIFIC': 'type_specific',
    'DISTRICT': 'district', 
    'PSA_NUM':'district_psa'
})

# List of columns to include, including the geometry column
columns_schools_with_geometry = [
    'school_id',
    'school_name',
    'school_name_label',
    'street_address',
    'zip_code',
    'phone_number',
    'grade_level',
    'grade_org',
    'type',
    'type_specific',
    'geometry',  
    'district', 
    'district_psa'
]

# Create 'school_info_df with the specified columns
school_info_df = school_with_psa_gdf[columns_schools_with_geometry]

# Drop rows where 'school_id' is None
school_info_df = school_info_df[school_info_df['school_id'].notna()]

# Drop rows with any NaN values
school_info_df = school_info_df.dropna(how='any')

# Remove duplicates based on 'school_id'
school_info_df = school_info_df.drop_duplicates(subset=['school_id'], keep='first')

# Strip unwanted characters and convert to integer
school_info_df['school_id'] = school_info_df['school_id'].astype(str).str.strip().str.replace(r'\r\n', '', regex=True).astype(int)

# Reset index for a clean index
school_info_df = school_info_df.reset_index(drop=True)

# Define the output file path
output_file_path = 'jupyter_outputs/School_Info.csv'

# Save the DataFrame to a CSV file
school_info_df.to_csv(output_file_path, index=False)

print(f"DataFrame saved to {output_file_path}")
print("-" * 60)

school_info_df.info()

DataFrame saved to jupyter_outputs/School_Info.csv
------------------------------------------------------------
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   school_id          322 non-null    int32   
 1   school_name        322 non-null    object  
 2   school_name_label  322 non-null    object  
 3   street_address     322 non-null    object  
 4   zip_code           322 non-null    object  
 5   phone_number       322 non-null    object  
 6   grade_level        322 non-null    object  
 7   grade_org          322 non-null    object  
 8   type               322 non-null    int64   
 9   type_specific      322 non-null    object  
 10  geometry           322 non-null    geometry
 11  district           322 non-null    object  
 12  district_psa       322 non-null    object  
dtypes: geometry(1), int32(1), int64(1),

In [19]:
# Read the GeoJSON file into a GeoDataFrame
geo_police_gdf = gpd.read_file('data/Police_Stations.geojson')
print("-" * 60)

geo_police_gdf.info()

------------------------------------------------------------
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   OBJECTID          24 non-null     int64   
 1   DISTRICT_NUMBER   24 non-null     int64   
 2   LOCATION          24 non-null     object  
 3   TELEPHONE_NUMBER  24 non-null     object  
 4   RULEID            24 non-null     object  
 5   geometry          24 non-null     geometry
dtypes: geometry(1), int64(2), object(3)
memory usage: 1.2+ KB


In [20]:
# Rename the specified columns
geo_police_gdf = geo_police_gdf.rename(columns={
    'OBJECTID': 'object_id', 
    'DISTRICT_NUMBER': 'district_number', 
    'LOCATION': 'location',
    'TELEPHONE_NUMBER': 'phone_number',
    'RULEID': 'zip_code'
})

geo_police_gdf

,object_id,district_number,location,phone_number,zip_code,geometry
0,1,7,Bustleton Ave & Bowler St,686-3070,Rule_1,POINT (-75.03270 40.09098)
1,2,14,Haines St & Germantown Ave,686-3140,Rule_1,POINT (-75.17700 40.03843)
2,3,8,Academy Rd & Red Lion Rd,686-3080,Rule_1,POINT (-74.99730 40.08011)
3,4,2,Harbison Ave & Levick St,686-3020,Rule_1,POINT (-75.06325 40.03149)
4,5,35,N Broad St & Champlost St,686-3350,Rule_1,POINT (-75.14365 40.04450)
5,6,15,Harbison Ave & Levick St,686-3150,Rule_1,POINT (-75.06325 40.03149)
6,7,5,Ridge Ave & Cinnaminson St,686-3050,Rule_1,POINT (-75.22440 40.04020)
7,8,39,22nd St & Hunting Park Ave,686-3390,Rule_1,POINT (-75.16425 40.01090)
8,9,25,3901 Whitaker Ave,686-3250,Rule_1,POINT (-75.12249 40.00874)
9,10,22,17th St & Montgomery Ave,686-3220,Rule_1,POINT (-75.16240 39.98079)


In [21]:
# Create a list of zip codes for philly schools and police
zip_codes = [
    19115,
    19144,
    19114,
    19149,
    19141,
    19149,
    19128,
    19140,
    19124,
    19121,
    19124,
    19125,
    19151,
    19104,
    19130,
    19107,
    19143,
    19146,
    19147,
    19142,
    19145,
    19130,
    19106,
    19104
]

# Add the 'zip_code' column to the DataFrame
geo_police_gdf['zip_code'] = zip_codes[:len(geo_police_gdf)]

police_location_df = pd.DataFrame({
    'district': geo_police_gdf['district_number'],
    'location': geo_police_gdf['location'],  
    'zip_code': geo_police_gdf['zip_code'],
    'phone_number': geo_police_gdf['phone_number'],
    'geometry': geo_police_gdf['geometry']
})

# Remove rows where the 'phone_number' column is empty
police_location_df = police_location_df[police_location_df['phone_number'].str.strip() != '']

# Sort the dataframe by the 'district' column in ascending order
police_location_df = police_location_df.sort_values(by='district', ascending=True).reset_index(drop=True)

# Make an output for the new dataset
output_file_path= 'jupyter_outputs/Police_Location.csv'

# Save the DataFrame to a CSV file
police_location_df.to_csv(output_file_path, index=False)

print(f"DataFrame saved to {output_file_path}")

police_location_df

DataFrame saved to jupyter_outputs/Police_Location.csv


,district,location,zip_code,phone_number,geometry
0,1,24th St & Wolf St,19145,686-3010,POINT (-75.18693 39.92407)
1,2,Harbison Ave & Levick St,19149,686-3020,POINT (-75.06325 40.03149)
2,3,11th St & Wharton St,19147,686-3030,POINT (-75.16229 39.93345)
3,5,Ridge Ave & Cinnaminson St,19128,686-3050,POINT (-75.22440 40.04020)
4,6,11th St & Winter St,19107,686-3060,POINT (-75.15719 39.95684)
5,7,Bustleton Ave & Bowler St,19115,686-3070,POINT (-75.03270 40.09098)
6,8,Academy Rd & Red Lion Rd,19114,686-3080,POINT (-74.99730 40.08011)
7,9,20th St & Pennsylvania Ave,19130,686-3090,POINT (-75.17159 39.96080)
8,12,65th St & Woodland Ave,19142,686-3120,POINT (-75.23434 39.92556)
9,14,Haines St & Germantown Ave,19144,686-3140,POINT (-75.17700 40.03843)


In [22]:
police_location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   district      21 non-null     int64   
 1   location      21 non-null     object  
 2   zip_code      21 non-null     int64   
 3   phone_number  21 non-null     object  
 4   geometry      21 non-null     geometry
dtypes: geometry(1), int64(2), object(2)
memory usage: 968.0+ bytes


### Creating a Police "Police Districts" DataFrame and Exporting for Use in Database

In [23]:
police_districts_df = pd.merge(district_psa_df, police_location_df, on='district', how='left')

police_districts_df

,district_psa,district,police_service_area,location,zip_code,phone_number,geometry
0,011,1,1,24th St & Wolf St,19145,686-3010,POINT (-75.18693 39.92407)
1,012,1,2,24th St & Wolf St,19145,686-3010,POINT (-75.18693 39.92407)
2,021,2,1,Harbison Ave & Levick St,19149,686-3020,POINT (-75.06325 40.03149)
3,022,2,2,Harbison Ave & Levick St,19149,686-3020,POINT (-75.06325 40.03149)
4,023,2,3,Harbison Ave & Levick St,19149,686-3020,POINT (-75.06325 40.03149)
...,...,...,...,...,...,...,...
63,352,35,2,N Broad St & Champlost St,19141,686-3350,POINT (-75.14365 40.04450)
64,353,35,3,N Broad St & Champlost St,19141,686-3350,POINT (-75.14365 40.04450)
65,391,39,1,22nd St & Hunting Park Ave,19140,686-3390,POINT (-75.16425 40.01090)
66,392,39,2,22nd St & Hunting Park Ave,19140,686-3390,POINT (-75.16425 40.01090)


In [24]:
# Make an output for the new dataset
output_file_path= 'jupyter_outputs/Police_Districts.csv'

# Save the DataFrame to a CSV file
police_districts_df.to_csv(output_file_path, index=False)

print(f"DataFrame saved to {output_file_path}")

police_districts_df.info()

DataFrame saved to jupyter_outputs/Police_Districts.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   district_psa         68 non-null     object  
 1   district             68 non-null     int64   
 2   police_service_area  68 non-null     object  
 3   location             68 non-null     object  
 4   zip_code             68 non-null     int64   
 5   phone_number         68 non-null     object  
 6   geometry             68 non-null     geometry
dtypes: geometry(1), int64(2), object(4)
memory usage: 3.8+ KB


In [25]:
# File Path to Total Crimes and Elementray School Data for Police Service Area
comparison_data_to_load = Path("SQL Outputs\Total_Crimes_ES_PSA.csv")

# Read Police Incidents Data File and store into Pandas DataFrames
comparison_data = pd.read_csv(comparison_data_to_load)

# Check DataFrame was created
comparison_data.head()

,district_psa,average_ela_proficiency,average_math_proficiency,average_percent_attending_90
0,11,34.70,28.000,54.20
1,21,26.00,19.700,62.75
2,23,37.40,30.625,70.40
3,33,46.55,41.400,73.90
4,72,62.20,54.500,81.00


In [26]:
# Calculate correlation matrix
correlation_matrix = comparison_data.corr()
correlation_matrix

,district_psa,average_ela_proficiency,average_math_proficiency,average_percent_attending_90
district_psa,1.000000,-0.403322,-0.441955,-0.325017
average_ela_proficiency,-0.403322,1.000000,0.954634,0.790114
average_math_proficiency,-0.441955,0.954634,1.000000,0.774163
average_percent_attending_90,-0.325017,0.790114,0.774163,1.000000


In [29]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the relationship between ELA Proficiency and Total Crimes
sns.regplot(x='average_ela_proficiency', y='total_crimes', data=comparison_data)
plt.title('ELA Proficiency vs Total Crimes')
plt.show()


KeyError: 'total_crimes'